In [ ]:
import os
from datetime import datetime
from urllib.parse import unquote, urlparse
import pandas as pd

import requests

current_year = datetime.now().year

from function.scraper import *

In [2]:
with open("irrelevant_domain.txt", "r") as file:
    domains_from_file = [line.strip() for line in file.readlines()]

In [3]:
# Load company list
company_df = pd.read_excel("rawdata/240209_ISA_ClientList_TM_V01.xlsx")
company_list = company_df["Company Name"].to_list()


search_companies = company_list[0:100]

# Define variables
search_year = "2022"
api_key = os.getenv("google_search_apikey")
cse_id = os.getenv("google_search_cseid")

search_and_save_results(
    search_companies = search_companies,
    search_year = search_year,
    api_key = api_key,
    cse_id = cse_id,
    search_query = "sustainability report pdf",
    num_results = 5,
    data_directory = "data/WS_ISA",
    save_to = "searchresult/isa_report_url.csv",
    load_from = "searchresult/isa_report_url.csv")

# clean up irrelevant domains
df_isa = pd.read_csv("searchresult/isa_report_url.csv", index_col=0)
df_isa["company_url"] = df_isa["url"].apply(extract_domain)
df_isa = df_isa[
    ~df_isa["company_url"].str.contains("|".join(domains_from_file), case=False)
]

# get a relevant score between the company and the url
df_isa["score"] = df_isa.apply(
    lambda row: name_match(row["company"], row["url"]), axis=1
)
df_isa = df_isa[
    df_isa.score > 0.07
]  # define a threshold, this is manually set by now by exploring the data and quantiles
df_isa["encoded_url"] = df_isa["url"].apply(unquote)
df_isa["year"] = df_isa["encoded_url"].apply(extract_year)
df_isa["year"] = df_isa["year"].fillna("No year from report url")
df_isa["year"] = df_isa.apply(
    lambda row: extract_year(row["encoded_url"], current_year), axis=1
)
df_isa.to_csv("searchresult/isa_report_url.csv")